In [1]:
import numpy as np
from numpy.ma.core import shape
import io

from swinArm.datamodule import vocab
from swinArm.lit_swinPreArm import LitSwinPreARM
from torchvision.transforms import ToTensor, Normalize, Resize, Compose
import torch
from PIL import Image


from IPython.display import display

In [4]:
ckpt = '../pretrain_model/lasted-v1.ckpt'

In [2]:
comp = Compose([
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    Resize((224, 224)),
])

In [6]:
model = LitSwinPreARM.load_from_checkpoint(ckpt)
model = model.eval()
device = torch.device("cpu")
model = model.to(device)


In [11]:
model

LitSwinPreARM(
  (model): SwinPreARM(
    (encoder): SwinV1Encoder(
      (swinv1): SwinTransformer(
        (patch_embed): PatchEmbed(
          (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
          (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        )
        (layers): Sequential(
          (0): SwinTransformerStage(
            (downsample): Identity()
            (blocks): Sequential(
              (0): SwinTransformerBlock(
                (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
                (attn): WindowAttention(
                  (qkv): Linear(in_features=96, out_features=288, bias=True)
                  (attn_drop): Dropout(p=0.2, inplace=False)
                  (proj): Linear(in_features=96, out_features=96, bias=True)
                  (proj_drop): Dropout(p=0.1, inplace=False)
                  (softmax): Softmax(dim=-1)
                )
                (drop_path1): Identity()
                (norm2): LayerNorm((96

In [15]:
for param in model.parameters():
    param.requires_grad = True
model.summarize()

C:\Users\trung\miniconda3\envs\swinarm\lib\site-packages\pytorch_lightning\core\lightning.py:1727: LightningDeprecationWarning: Argument `mode` in `LightningModule.summarize` is deprecated in v1.4 and will be removed in v1.6. Use `max_depth=1` to replicate `mode=top` behavior.
  f"Argument `mode` in `LightningModule.summarize` is deprecated in v1.4"

  | Name             | Type            | Params
-----------------------------------------------------
0 | model            | SwinPreARM      | 30.9 M
1 | exprate_recorder | ExpRateRecorder | 0     
-----------------------------------------------------
30.9 M    Trainable params
0         Non-trainable params
30.9 M    Total params
123.797   Total estimated model params size (MB)


  | Name             | Type            | Params
-----------------------------------------------------
0 | model            | SwinPreARM      | 30.9 M
1 | exprate_recorder | ExpRateRecorder | 0     
-----------------------------------------------------
30.9 M    Trainable params
0         Non-trainable params
30.9 M    Total params
123.797   Total estimated model params size (MB)

In [48]:
img = Image.open('original.png').convert('RGB')
display(img)

SyntaxError: invalid syntax (581394333.py, line 1)

In [ ]:
img = np.array(img)

In [ ]:
img = comp(img)
mask = torch.zeros_like(img[-1,:,:], dtype=torch.bool)
hyp = model.approximate_joint_search(img.unsqueeze(0), 
                                     mask.unsqueeze(0))[0]
pred_latex = vocab.indices2label(hyp.seq)

In [ ]:
print(pred_latex)